In [1]:
# Install PyMongo
!pip install pymongo

In [2]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure
from bson.objectid import ObjectId

In [4]:
# MongoDB connection details (admin credentials)
MONGO_URI_ADMIN = "mongodb://localhost:27017/"

# Function to create database and user
def setup_mongodb():
    try:
        # Connect to MongoDB as admin
        client = MongoClient(MONGO_URI_ADMIN)
        db = client.bookbazaar_reviews  # Create the database

        # Create a user for the database
        db.command("createUser", "bookbazaar_user", pwd="userpassword", roles=["readWrite"])
        print("Database 'bookbazaar_reviews' and user 'bookbazaar_user' created successfully!")
    except OperationFailure as e:
        print(f"Failed to create database or user: {e}")
    except ConnectionFailure as e:
        print(f"Failed to connect to MongoDB: {e}")

# Run the setup
setup_mongodb()

Database 'bookbazaar_reviews' and user 'bookbazaar_user' created successfully!


In [5]:
# MongoDB connection details (user credentials)
MONGO_URI = "mongodb://localhost:27017/bookbazaar_reviews"

# Function to connect to MongoDB
def connect_to_mongodb():
    try:
        # Create a connection to MongoDB
        client = MongoClient(MONGO_URI)
        # Ping the server to confirm the connection
        client.admin.command('ping')
        print("Successfully connected to MongoDB!")
        return client
    except ConnectionFailure as e:
        print(f"Failed to connect to MongoDB: {e}")
        return None

# Connect to MongoDB
client = connect_to_mongodb()

Successfully connected to MongoDB!


In [8]:
# Function to add a new review
def add_review(book_id, user_id, rating, comment):
    if client:
        db = client.bookbazaar_reviews
        reviews_collection = db.reviews
        review = {
            "book_id": book_id,
            "user_id": user_id,
            "rating": rating,
            "comment": comment
        }
        result = reviews_collection.insert_one(review)
        print(f"Review added with ID: {result.inserted_id}")
    else:
        print("No MongoDB connection.")

# Example usage
add_review(book_id=1, user_id=1, rating=5, comment="Good book!")
add_review(book_id=2, user_id=2, rating=7, comment="Great book!")
add_review(book_id=3, user_id=3, rating=9, comment="Amazing book!")

Review added with ID: 6785095d1e3ffbdb680e6965
Review added with ID: 6785095d1e3ffbdb680e6966
Review added with ID: 6785095d1e3ffbdb680e6967


In [10]:
# Function to retrieve reviews for a specific book
def get_reviews_for_book(book_id):
    if client:
        db = client.bookbazaar_reviews
        reviews_collection = db.reviews
        reviews = list(reviews_collection.find({"book_id": book_id}))
        print(f"Reviews for book {book_id}:")
        for review in reviews:
            print(review)
    else:
        print("No MongoDB connection.")

# Example usage
get_reviews_for_book(book_id=1)
get_reviews_for_book(book_id=2)
get_reviews_for_book(book_id=3)

Reviews for book 1:
{'_id': ObjectId('678508e91e3ffbdb680e6964'), 'book_id': 1, 'user_id': 1, 'rating': 5, 'comment': 'Great book!'}
{'_id': ObjectId('6785095d1e3ffbdb680e6965'), 'book_id': 1, 'user_id': 1, 'rating': 5, 'comment': 'Good book!'}
Reviews for book 2:
{'_id': ObjectId('6785095d1e3ffbdb680e6966'), 'book_id': 2, 'user_id': 2, 'rating': 7, 'comment': 'Great book!'}
Reviews for book 3:
{'_id': ObjectId('6785095d1e3ffbdb680e6967'), 'book_id': 3, 'user_id': 3, 'rating': 9, 'comment': 'Amazing book!'}


In [11]:
# Function to update a review
def update_review(review_id, new_rating, new_comment):
    if client:
        db = client.bookbazaar_reviews
        reviews_collection = db.reviews
        result = reviews_collection.update_one(
            {"_id": review_id},
            {"$set": {"rating": new_rating, "comment": new_comment}}
        )
        if result.modified_count > 0:
            print(f"Review {review_id} updated successfully.")
        else:
            print(f"No review found with ID {review_id}.")
    else:
        print("No MongoDB connection.")

# Example usage (replace '...' with the actual _id of a review)
update_review(review_id=ObjectId("6785095d1e3ffbdb680e6965"), new_rating=4, new_comment="Bad book!")

Review 6785095d1e3ffbdb680e6965 updated successfully.


In [12]:
get_reviews_for_book(book_id=1)
get_reviews_for_book(book_id=2)
get_reviews_for_book(book_id=3)

Reviews for book 1:
{'_id': ObjectId('678508e91e3ffbdb680e6964'), 'book_id': 1, 'user_id': 1, 'rating': 5, 'comment': 'Great book!'}
{'_id': ObjectId('6785095d1e3ffbdb680e6965'), 'book_id': 1, 'user_id': 1, 'rating': 4, 'comment': 'Bad book!'}
Reviews for book 2:
{'_id': ObjectId('6785095d1e3ffbdb680e6966'), 'book_id': 2, 'user_id': 2, 'rating': 7, 'comment': 'Great book!'}
Reviews for book 3:
{'_id': ObjectId('6785095d1e3ffbdb680e6967'), 'book_id': 3, 'user_id': 3, 'rating': 9, 'comment': 'Amazing book!'}


In [13]:
# Function to delete a review
def delete_review(review_id):
    if client:
        db = client.bookbazaar_reviews
        reviews_collection = db.reviews
        result = reviews_collection.delete_one({"_id": review_id})
        if result.deleted_count > 0:
            print(f"Review {review_id} deleted successfully.")
        else:
            print(f"No review found with ID {review_id}.")
    else:
        print("No MongoDB connection.")

# Example usage (replace '...' with the actual _id of a review)
delete_review(review_id=ObjectId("678508e91e3ffbdb680e6964"))

Review 678508e91e3ffbdb680e6964 deleted successfully.


In [14]:
get_reviews_for_book(book_id=1)
get_reviews_for_book(book_id=2)
get_reviews_for_book(book_id=3)

Reviews for book 1:
{'_id': ObjectId('6785095d1e3ffbdb680e6965'), 'book_id': 1, 'user_id': 1, 'rating': 4, 'comment': 'Bad book!'}
Reviews for book 2:
{'_id': ObjectId('6785095d1e3ffbdb680e6966'), 'book_id': 2, 'user_id': 2, 'rating': 7, 'comment': 'Great book!'}
Reviews for book 3:
{'_id': ObjectId('6785095d1e3ffbdb680e6967'), 'book_id': 3, 'user_id': 3, 'rating': 9, 'comment': 'Amazing book!'}
